In [206]:
#!/usr/bin/env python
import os
from witb.utils import ioutils as io
from witb.globals import DATA_DIR
import time  
import nltk
import csv
import re, string
import collections
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

In [321]:
stop_words = set(stopwords.words('english'))

In [2]:
DATA_DIR="/home/sasha/Documents/MilaPostDoc/Python/CommonCrawl"

In [301]:
blacklistcsv='wordlists/porn_blacklist_filtered.csv'

In [327]:
bigrams='wordlists/porn_bigrams_filtered.csv'

In [328]:
bigramlist = []
with open(bigrams, newline='') as inputfile:
    for row in csv.reader(inputfile):
        bigramlist.append(tuple(row[0].split()))
print(len(bigramlist))

758


In [299]:
bigramlist

[('hairy', 'pussy'),
 ('wet', 'pussy'),
 ('black', 'pussy'),
 ('tight', 'pussy'),
 ('pussy', 'porn'),
 ('sex', 'videos'),
 ('pussy', 'sex'),
 ('shaved', 'pussy'),
 ('girl', 'pussy'),
 ('pussy', 'pics'),
 ('eating', 'pussy'),
 ('pussy', 'licking'),
 ('her', 'pussy'),
 ('pussy', 'free'),
 ('licking', 'pussy'),
 ('pink', 'pussy'),
 ('teen', 'pussy'),
 ('big', 'pussy'),
 ('pussy', 'fuck'),
 ('eat', 'pussy'),
 ('my', 'pussy'),
 ('naked', 'pussy'),
 ('fat', 'pussy'),
 ('pussy', 'and'),
 ('girl', 'sex'),
 ('having', 'sex'),
 ('lesbian', 'pussy'),
 ('porn', 'movies'),
 ('fucking', 'pussy'),
 ('pussy', 'videos'),
 ('sex', 'pussy'),
 ('pussy', 'video'),
 ('skinny', 'teen'),
 ('girl', 'porn'),
 ('in', 'pussy'),
 ('porn', 'videos'),
 ('white', 'pussy'),
 ('free', 'porn'),
 ('sex', 'video'),
 ('and', 'pussy'),
 ('sex', 'with'),
 ('anal', 'sex'),
 ('sex', 'movies'),
 ('pussy', 'pictures'),
 ('little', 'pussy'),
 ('pussy', 'lips'),
 ('girls', 'fucking'),
 ('pussy', 'fucked'),
 ('pussy', 'eating'),
 (

In [302]:
blacklist = []
with open(blacklistcsv, newline='') as inputfile:
    for row in csv.reader(inputfile):
        blacklist.append(row[0])
print(len(blacklist))

3568


In [3]:
filenames = os.listdir(DATA_DIR)
filename = os.path.join(DATA_DIR, filenames[0])

data = io.get_entries(io.read_wet_file(filename))
parsed_data = io.parse_data(data)  # Generator.

docs = []
start_time = time.time()
for doc in parsed_data:
    docs.append(doc)

In [307]:
for doc in docs:
    if doc.url=='www.twinkpornvideo.com':
        testdoc=doc.content

In [308]:
testdoc

"View Park-Windsor Hills ts hookers, hook up - Twink Porn ✔️\nTWINK PORN\nHookers\nUnited States\nCalifornia\nView Park-Windsor Hills\nTs hookers View Park-Windsor Hills\nAmanda carey trans 22x4 passing through View Park-Windsor Hills 15 th for curious straight men\nTs hook up View Park-Windsor Hills\nHello everyone, I am Amanda Carey Brazilian from passag in View Park-Windsor Hills 15 EME I am active and passive. Call me for more information thanks to all of you kisses.\nParty duo your best option to make a rich trio\nView Park-Windsor Hills escort incall\nOnly a couple of scorts a beautiful girl trans gifted i morbid i a handsome guy very gifted we are very party we like the morbid and much sex we are dominant i very affectionate at the time\nKaren trans 20 cm 100% real\nView Park-Windsor Hills ts hook up\nHello darling, 200% real and NO SURPRISES ON PLACE I'm Karen, Pretty new latina in the corner. I offer you to spend divine moments with me, I am expert and gentle for beginners and

In [167]:
#Simple word search
start = time.time()
pornsites={}
for doc in docs:
    pwds=len([w for w in word_tokenize(doc.content) if w in blacklist])
    pornsites[doc.url]=pwds
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

KeyboardInterrupt: 

In [303]:
#Bigram search with counts
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=len(match)
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

Number of pages counted:  12903
Duration:  0:04:14.038242


In [ ]:
#Bigram search with counts and list of tuples found (for debugging)
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=[len(match), str(match)]
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

In [ ]:
{k: v for k, v in sorted(pornsites.items(), key=lambda item: item[1], reverse=True)}

In [319]:
len({k: v for k, v in pornsites.items() if v > 3})

167

In [316]:
167.00/12903.00

0.01294272649771371